In [358]:
import os
import json
import cohere
import re

from langchain_openai import AzureOpenAIEmbeddings
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents._generated.models import QueryCaptionResult

from openai import AzureOpenAI
import tiktoken

from dotenv import load_dotenv

load_dotenv()

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

class AzureEmbeddings:

    def __init__(self):
        pass

    @staticmethod
    def get_embedding():
        return AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

    @staticmethod
    def generate_embeddings(content: str):
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

        doc_result = embeddings.embed_documents([content])

        return doc_result[0]

openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"), 
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    )

embeddings_client = AzureEmbeddings()
store_search_url: str = f'https://{os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME')}.search.windows.net'
search_client = SearchClient(
            store_search_url, os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME"),
            AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"))
        )

co = cohere.Client(api_key=os.getenv('COHERE_API_KEY'))

In [359]:
### 🙎‍♂️ Definir la pregunta

# user_input = "¿Qué dice el artículo 103 del Código Penal?"  # 1
# user_input = "¿Cuáles son las causales de agravación del homicidio?"  # 2
# user_input = "Si A le roba el celular a B, ¿qué delito comete?"  # 3
# user_input = "¿Cuál es la diferencia entre dolo eventual y culpa con representación?"  # 4
# user_input = "¿Cuáles son las formas de autoría en derecho penal?"  # 5
# user_input = "¿Cuándo se consuma el hurto?"  # 6
# user_input = "¿Las personas jurídicas pueden ser sujetos pasivos de los delitos de injuria y calumnia?"  # 7
# user_input = "¿Cuál es el término que tiene la Fiscalía para formular acusación?"  # 8
# user_input = "¿Se puede archivar una investigación cuando se encuentra que el procesado actuó en legítima defensa?"  # 9
# user_input = "¿Se puede precluir una investigación antes de formular imputación?"  # 10
# user_input = "¿Cuáles son las causales de ausencia de responsabilidad en el derecho penal?"  # 11
# user_input = "¿Cuáles son las causales genéricas de atenuación punitiva?"  # 12
# user_input = "¿Qué se entiende por “documento” en el derecho penal?"  # 13
# user_input = "¿Cuáles son las causales de agravación de la estafa?"  # 14
# user_input = "¿Se pueden realizar preguntas sugestivas en el contrainterrogatorio del juicio oral?" #15
# user_input = "¿Un juez debe ordenar la captura del procesado cuando anuncie sentido de fallo condenatorio?" #16
# user_input = "¿Cuáles son las fuentes de posición de garante según el código penal?" #17
# user_input = "¿La búsqueda selectiva en base de datos requiere control previo?" #18
# user_input = "¿Cuándo procede la captura en flagrancia?" #19
# user_input = "Si alguien falsifica un documento y lo usa como medio para engañar a otra persona y llevarlo a que le pague algo que no le debe, ¿qué delito(s) comete?" #20

# user_input = "Cuál es la sanción por no presentar la declaración de renta a timepo?"
# user_input = "Qué es el derecho a la vida?"
# user_input = "En qué artículo de la constitución está consagrado el derecho a la vida?"
# user_input = "En qué artículo del código penal se contempla la definición de documento?"
# user_input = "¿Qué dice el artículo 14 del código penal?"
# user_input = "¿Qué dice el artículo 14 de la ley 599 del 2000?"
# user_input = "¿Dónde se considera realizada la conducta punible?"
# user_input = "¿Qué dice el artículo 241 del código penal?"
# user_input = "¿Qué se entiende por “documento” en el código penal?"
# user_input = "¿Qué dice el artículo 249 del código penal?"
# user_input = "¿Qué dice el artículo 221 del código penal?"
# user_input = "¿En qué artículo se encuentra el delito de calumnia?"
# user_input = "Si B imputa falsamente a A un delito que no ha cometido ¿En qué conducta punible incurre?"
# user_input = "En qué delito incurre una persona que entre sin autorización en un sistema informático?"
# user_input = "Daniel Andrés Fúquenes Barriga, en su condición de auxiliar de la justicia y secuestre, recaudó una suma de dinero por concepto de arrendamiento, específicamente $684,000 entre febrero y julio de 2015. Este dinero debía ser entregado a su dueño o poseedor, pero Fúquenes Barriga retuvo la suma para sí mismo, incumpliendo con la obligación de devolverla. Este acto de retención y apropiación del dinero, que se le había confiado por un título no traslativo de dominio, ¿Qué delito cometió?"
# user_input = "¿Qué delito comtió una persona que adquirió un bien cuyo origen inmediato es una actividad ilícita?"
# user_input = "¿Qué dice el artículo 323 del código penal?"
# user_input = "¿Qué artículo contiene el delito de lavado de activos en el código penal?"
# user_input = "¿Qué delito comete un juez que profiere una sentencia manifiestamente contraria a la ley?"
# user_input = "Si A dispara a B en Barrranquilla pero B se muere en Cartagena, ¿Dónde se entiende realizado el homicidio?"
# user_input = "Según el artículo 14 del código penal si A dispara a B en Barrranquilla pero B se muere en Cartagena, ¿Dónde se entiende realizado el homicidio?"
# user_input = "¿Qué se entiende por servido público en código penal?"
user_input = "Si A falsifica un cheque y lo utiliza para comprarle un televisor a B, ¿qué delito comete?"

print("🧑 User input:")
print(user_input)

🧑 User input:
Si A falsifica un cheque y lo utiliza para comprarle un televisor a B, ¿qué delito comete?


In [360]:
### 💠 Vectorizar la pregunta

user_input_vectors = embeddings_client.generate_embeddings(content=user_input)
print("🧑 User input:")
print(user_input)

🧑 User input:
Si A falsifica un cheque y lo utiliza para comprarle un televisor a B, ¿qué delito comete?


In [361]:
### 👀 Búsqueda inicial con esa pregunta


def search_initial_chunks():
    return search_client.search(
        search_text=user_input,
        vector_queries=[
            {
                "vector": user_input_vectors,
                "k": 50,
                "fields": "content_vector",
                "kind": "vector",
                "exhaustive": True,
            }
        ],
        top=50,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="ariel-alberto",
        query_caption="extractive|highlight-false",
        scoring_profile="ariel-alberto",
    )


results_list = list(search_initial_chunks())

In [362]:
### 🗂️ Formatear los resultados

print("🧑 User input:")
print(user_input)

documents = []
printer_documents = []

for document in results_list:
    captions: QueryCaptionResult = document["@search.captions"]
    captions_text = " // ".join([caption.text for caption in captions])
    doc_formatted = {
        # "score": document["@search.score"],
        # "rerank": document["@search.reranker_score"],
        # "captions": captions_text,
        "título": document["title"],
        "autor": document["author"],
        "tema_legal": document["keywords"],
        "tipo_de_documento": document["category"],
        "contenido": document["content"],
    }
    documents.append(doc_formatted)
    printer_documents.append(json.dumps(doc_formatted, indent=4))
    print(json.dumps(doc_formatted, indent=4))

🧑 User input:
Si A falsifica un cheque y lo utiliza para comprarle un televisor a B, ¿qué delito comete?
{
    "t\u00edtulo": "CSJ - SP34199(26-05-2010)",
    "autor": "Corte Suprema de Justicia",
    "tema_legal": "Penal",
    "tipo_de_documento": "Jurisprudencia",
    "contenido": "\"La Bolsa del Pr\u00e9stamo\".\nAl recaer la presunta acci\u00f3n falsaria en uno de los componentes de !a\nfactura, como lo es la identificaci\u00f3n serial del aparato adquirido, t\ncomportamiento tiene como presupuesto el car\u00e1cter genuino inici\ndel documento. En otras palabras, para los operadores judiciales Vs\nprocesados s\u00ed adquirieron un televisor, el mismo les fue hurtado y i\nfactura arrimada demuestra [a leg\u00edtima adquisici\u00f3n del mismo.\nConforme a ello, la acci\u00f3n desplegada por sus defendidos, consistente\nen adicionar a la factura la identificaci\u00f3n del aparato, \"no es de\nentidad requerida por el legislador para considerar colmada ii\nconducta punible de fraude pr

In [363]:
### 🎖️ Rerankear los resultados

print("🧑 User input:")
print(user_input)

reranked_docs = co.rerank(
    model="rerank-multilingual-v3.0",
    top_n=5,
    query=user_input,
    documents=documents,
    return_documents=True,
    rank_fields=["contenido"],
)

🧑 User input:
Si A falsifica un cheque y lo utiliza para comprarle un televisor a B, ¿qué delito comete?


In [364]:
### 🗂️ Formatear el rerankeo

print("🧑 User input:")
print(user_input)

reranked_list = reranked_docs.results

reranked_documents = []

for doc in reranked_list:
    doc_formatted = {
        # "relevancia": doc.relevance_score,
        "título": doc.document.título,
        "autor": doc.document.autor,
        "tema_legal": doc.document.tema_legal,
        "tipo_de_documento": doc.document.tipo_de_documento,
        "contenido": doc.document.contenido,
    }
    reranked_documents.append(doc_formatted)
    print(json.dumps(doc_formatted, indent=4))

🧑 User input:
Si A falsifica un cheque y lo utiliza para comprarle un televisor a B, ¿qué delito comete?
{
    "t\u00edtulo": "CASTRO CUENCA - Manual de Derecho penal Parte Especial - Tomo I - 2019",
    "autor": "Carlos Castro Cuenca",
    "tema_legal": "Penal",
    "tipo_de_documento": "Doctrina",
    "contenido": "C\u00f3digo Penal. Al emitirse un cheque, lo que implica su creaci\u00f3n y entrega al\nles mensuales vigentes.\ntenedor, se est\u00e1 procediendo a un pago que, en caso de no hacerse efectivo por\ncausas ajenas a este, cuando menos pone en peligro su patrimonio econ\u00f3mico, \"E;1l a misma pena i~~urrir\u00e1 el que en loter\u00eda, rifa o juego, obtenga provecho\ny en la mayor\u00eda de ocasiones, le genera una afectaci\u00f3n patrimonial. para s1 o p~a otros, vahendose de cualquier medio fraudulento para asegurar\nun deterrnmado resultado.\nEsta opini\u00f3n no se comparte por quienes sostienen esta tesis, pues consi\nderan que, en el caso del fraude mediante cheque, 

In [365]:
### 🔑 Definir system prompt y tools

# query_prompt = "Generate 1 or multiple search queries in Spanish based on what the user asked you. Each query should always contain at least one law, articule and/or document title. The query will be used in a hybrid search (semantic and full text) so keep it short but make sure to include all the main keywords and actions needed to give the necesary context to the query. In the query you should include at least one concept followed by one specific source. eg: 'homicidio artículo 103 ley 599 del 2000'. Don't include more than 1 source. Sometimes the user will tell you the exact source you should query, but if you can't find the source out of what the user asked you, you can use the 'Supporting Information' to identify sources that could help you generate a better query with the user question."
query_prompt = "Generate 1 or multiple search queries in Spanish based on what the user asked you. Each query should contain the whole idea and sources if they are needed. If the user question adresses a when, how or what information, you should keep that in the query because that will affect what you will get. The query will be used in a hybrid search (semantic and full text) so keep it short but make sure to include all the main keywords and infromation needed to give the necesary context to the query. Try to always include at least one concept followed by one specific source. Don't include more than 1 source. You can include the various ways of refering the same source eg: 'homicidio artículo 103 ley 599 del 2000 código penal'. Sometimes the user will tell you the exact source you should query, but if you can't find the source out of what the user asked you, you can use the 'Supporting Information' to identify sources that could help you generate a better query with the user question."

search_legal_info = {
    "type": "function",
    "function": {
        "name": "search_legal_info",
        "description": query_prompt,
        "parameters": {
            "type": "object",
            "strict": "true",
            "properties": {
                "search_query": {
                    "type": "string",
                    "description": "The text search query for supporting documents. eg: 'homicidio artículo 103 ley 599 del 2000', 'cuándo se consuma el hurto artículo 239 ley 599 del 2000'",
                },
                "search_type": {
                    "enum": ["exact", "interpret"],
                    "type": "string",
                    "description": "If the query is for and exact citation or for a more concept interpretation",
                },
                "number_of_chunks_needed": {
                    "type": "integer",
                    "description": "A number between 3 and 10 depending on how many documents of 500 tokens you want to retrieve out of this query",
                },
            },
            "required": ["search_query", "search_type", "number_of_chunks_needed"],
        },
    },
}
get_next_chunk = {
    "type": "function",
    "function": {
        "name": "get_next_chunk",
        "description": "Use this tool to get when an important sources comes incomplete or the content is cut off. This tool will help you retrieve the following section of that given source.",
        "parameters": {
            "type": "object",
            "properties": {
                "source_id": {
                    "type": "string",
                    "description": "the unique identifier of the chunk that is incomplete. eg:'20240719192458csjscpboletinjurisprudencial20181219pdf_chunk30'",
                },
            },
            "required": ["source_id"],
        },
    },
}

response_system_template = f'Eres Ariel, un asistente para la investigación legal. Sé lo más detallado y preciso posible en tus respuestas. Cita el máximo número de fuentes posible, sin salirte del tema. En caso de encontrar información contradictoria, señálala y sugiere una posible causa. Expresa toda la información que encuentres en las fuentes proporcionadas por el usuario. Solamente si la respuesta no está en las fuentes proporcionadas, responde "No encuentro información con esos términos, ¿puedes reformular tu consulta?". Las fuentes proporcionadas por el usuario son fragmentos de máximo 500 tokens del documento completo, así que en ocasionas las fuentes proporcionadas van a tener contenido incompleto o cortado, si ves que la fuente más importante no está completa o puede que el documento completo tenga más información relevante, ejecuta "get_next_chunk" para obtener el siguiente fragmento de esa fuente. Interpreta las fuentes dependiendo de su categoría: 1. Constitucional: Es la norma máxima y más importante establecida por el gobierno del país. 2. Legal: Es la que le sigue en importancia y son los decretos, códigos y leyes oficiales del país establecida por los ministerios. 3. Infralegal: Le sigue en importancia a la categoría Legal. 4. Jurisprudencia: Son los documentos de juicios reales y de cómo se ha ejercido la ley en el pasado. 5. Doctrina: Son documentos o libros escritos por autores del derecho que no son parte de la ley pero ayudan a darle una interpetación a la ley. Es Al incluir títulos en tu respuesta, usa formato html (ej: titulos, <strong>). Recibirás las fuente n un formato json donde encontrarás el contenido de la fuente, el título del documento, entre otra información de utilidad, siempre incluye el título de la fuente para cada hecho que uses en la respuesta. Todas las fuentes son PDFs. Utiliza corchetes para referenciar la fuente, por ejemplo [info1.pdf]. No combines fuentes, lista cada fuente por separado, por ejemplo [info1.txt] [info2.pdf]. El formato de algunas fuentes puede incluir acentos, puntos o guiones. No manipules los id de las fuentes ya que ellos son importantes para el funcionamiento de esta conversación. Escribe la respuesta en formato HTML, pero sin incluir los tags "```html" al principio o final principio de tu respuesta.'

messages = [
    {"role": "system", "content": response_system_template},
    {"role": "user", "content": user_input},
    {
        "role": "assistant",
        "content": f"Supporting Information: \n\n {reranked_documents}",
    },
]

print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "Eres Ariel, un asistente para la investigaci\u00f3n legal. S\u00e9 lo m\u00e1s detallado y preciso posible en tus respuestas. Cita el m\u00e1ximo n\u00famero de fuentes posible, sin salirte del tema. En caso de encontrar informaci\u00f3n contradictoria, se\u00f1\u00e1lala y sugiere una posible causa. Expresa toda la informaci\u00f3n que encuentres en las fuentes proporcionadas por el usuario. Solamente si la respuesta no est\u00e1 en las fuentes proporcionadas, responde \"No encuentro informaci\u00f3n con esos t\u00e9rminos, \u00bfpuedes reformular tu consulta?\". Las fuentes proporcionadas por el usuario son fragmentos de m\u00e1ximo 500 tokens del documento completo, as\u00ed que en ocasionas las fuentes proporcionadas van a tener contenido incompleto o cortado, si ves que la fuente m\u00e1s importante no est\u00e1 completa o puede que el documento completo tenga m\u00e1s informaci\u00f3n relevante, ejecuta \"get_next_chunk\" para

In [366]:
### 🤖 Generar query

frist_completion = openai_client.chat.completions.create(
    tools=[search_legal_info],
    messages=messages,
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0.0,
    n=1,
    tool_choice=search_legal_info,
)

In [367]:
### 🗂️ Formatear query

print(frist_completion)
print("===========")
print(frist_completion.choices[0].message.tool_calls)
print("===========")
query = json.loads(frist_completion.choices[0].message.tool_calls[0].function.arguments)
print(query)

ChatCompletion(id='chatcmpl-A5xP2FEG1MTVux0hA2cBOSsoxR4ns', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7WDMBwcZNcBGKKLSPwSXwqz8', function=Function(arguments='{"search_query":"falsificación de cheque y uso para comprar artículo 289 código penal","search_type":"exact","number_of_chunks_needed":3}', name='search_legal_info'), type='function')]), content_filter_results={})], created=1725983348, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_80a1bad4c7', usage=CompletionUsage(completion_tokens=31, prompt_tokens=3336, total_tokens=3367), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'seve

In [368]:
### 💠 Vectorizar el query

query_vectors = embeddings_client.generate_embeddings(content=query.get("search_query"))
print(query.get("search_query"))

falsificación de cheque y uso para comprar artículo 289 código penal


In [369]:
### 🔎 Búsqueda con query


def search_chunks():
    return search_client.search(
        search_text=query.get("search_query"),
        vector_queries=[
            {
                "vector": query_vectors,
                "k": 50,
                "fields": "content_vector",
                "kind": "vector",
                "exhaustive": True,
            }
        ],
        top=50,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="ariel-alberto",
        query_caption="extractive|highlight-false",
        scoring_profile="ariel-alberto",
    )


results_list = list(search_chunks())

In [370]:
### 🗂️ Formatear los resultados

print("🔎 Query:")
print(query.get("search_query"))

documents = []

for document in results_list:
    doc_formatted = {
        "id": document["id"],
        "título": document["title"],
        "autor": document["author"],
        "página": document["page"],
        "tema_legal": document["keywords"],
        "tipo_de_documento": document["category"],
        "contenido": document["content"],
    }
    documents.append(doc_formatted)
    print(json.dumps(doc_formatted, indent=4))

🔎 Query:
falsificación de cheque y uso para comprar artículo 289 código penal
{
    "id": "20240825093512csjcp085202156280pdf_chunk15",
    "t\u00edtulo": "CSJ - CP085-2021(56280)",
    "autor": "Corte Suprema de Justicia",
    "p\u00e1gina": 16,
    "tema_legal": "Penal",
    "tipo_de_documento": "Jurisprudencia",
    "contenido": "conmoci\u00f3n p\u00fablica o de un infortunio particular del\ndamnificado;\n3\u00ba Cuando se hiciere uso de ganz\u00faa, llave falsa u otro\ninstrumento semejante o de llave verdadera que\nhubiere sido substra\u00edda, hallada o retenida;\n4\u00ba Cuando se perpetrare con escalamiento.\n5\u00ba Cuando el hurto fuese de mercader\u00edas u otras cosas\nmuebles transportadas por cualquier medio y se\ncometiere entre el momento de su carga y el de su\ndestino o entrega, o durante las escalas que se\nrealizaren.\n6\u00ba Cuando el hurto fuere de veh\u00edculos dejados en la v\u00eda\np\u00fablica o en lugares de acceso p\u00fablico.\nTITULO XII\nDELITOS CONTRA

In [371]:
### 🎖️ Rerankear los resultados

print("🎖️ Rerank by:")
rerank_term = f"{user_input} ({query.get("search_query")})"
print(rerank_term)


reranked_docs = co.rerank(
    model="rerank-multilingual-v3.0",
    top_n=5,
    query=rerank_term,
    documents=documents,
    return_documents=True,
    rank_fields=["contenido"],
)

🎖️ Rerank by:
Si A falsifica un cheque y lo utiliza para comprarle un televisor a B, ¿qué delito comete? (falsificación de cheque y uso para comprar artículo 289 código penal)


In [372]:
### 🗂️ Formatear el rerankeo

print("🔎 Query:")
print(query.get("search_query"))

reranked_list = reranked_docs.results

reranked_documents = []

for doc in reranked_list:
    doc_formatted = {
        "relevancia": doc.relevance_score,
        "id": doc.document.id,
        "título": doc.document.título,
        "autor": doc.document.autor,
        "página": doc.document.autor,
        "tema_legal": doc.document.tema_legal,
        "tipo_de_documento": doc.document.tipo_de_documento,
        "contenido": doc.document.contenido,
    }
    reranked_documents.append(doc_formatted)
    print(json.dumps(doc_formatted, indent=4))

🔎 Query:
falsificación de cheque y uso para comprar artículo 289 código penal
{
    "relevancia": 0.9913892,
    "id": "20240813150354universidadexternadoleccionesdederechopenalparteespecialvol12011pdf_chunk616",
    "t\u00edtulo": "UNIVERSIDAD EXTERNADO - Lecciones de Derecho penal - parte especial vol 1 -2011",
    "autor": "Universidad Externado",
    "p\u00e1gina": "Universidad Externado",
    "tema_legal": "Penal",
    "tipo_de_documento": "Doctrina",
    "contenido": "falsificador\u2013 debe esperar un mes para que se efect\u00fae el traslado de fondos\nque se quiere afectar, deber\u00e1 admitirse que se trata de una sola falsificaci\u00f3n\ndocumental privada en el marco del art\u00edculo 289 del C. P.\nComo es evidente, ello puede presentar algunos problemas de orden\nprocesal, que en la pr\u00e1ctica deben resolverse atendiendo a la unidad de\nconducta t\u00edpica, si ella subsiste.\nSi el segundo acto no se produjere, es obvio que todo queda en tentativa\nde falsedad document

In [373]:
### 🔑 Armar mensajes

messages = [
    {"role": "system", "content": response_system_template},
    {"role": "user", "content": user_input},
    {
        "role": "assistant",
        "content": f"Chunks encontrados al consultar con el query '{query.get("search_query")}': \n\n {reranked_documents}",
    },
]

print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "Eres Ariel, un asistente para la investigaci\u00f3n legal. S\u00e9 lo m\u00e1s detallado y preciso posible en tus respuestas. Cita el m\u00e1ximo n\u00famero de fuentes posible, sin salirte del tema. En caso de encontrar informaci\u00f3n contradictoria, se\u00f1\u00e1lala y sugiere una posible causa. Expresa toda la informaci\u00f3n que encuentres en las fuentes proporcionadas por el usuario. Solamente si la respuesta no est\u00e1 en las fuentes proporcionadas, responde \"No encuentro informaci\u00f3n con esos t\u00e9rminos, \u00bfpuedes reformular tu consulta?\". Las fuentes proporcionadas por el usuario son fragmentos de m\u00e1ximo 500 tokens del documento completo, as\u00ed que en ocasionas las fuentes proporcionadas van a tener contenido incompleto o cortado, si ves que la fuente m\u00e1s importante no est\u00e1 completa o puede que el documento completo tenga m\u00e1s informaci\u00f3n relevante, ejecuta \"get_next_chunk\" para

In [374]:
### 🤖 Generar respuesta

second_completion = openai_client.chat.completions.create(
    tools=[search_legal_info, get_next_chunk],
    messages=messages,
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0.2,
    n=1,
    tool_choice="auto",
)

In [375]:
### 💬 Formatear respuesta

print(second_completion)
print("===========")
print(second_completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-A5xPFiKK8x8bBPAVD8YTHRCFu6X3h', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ixp6OBVCDMpeVsiMBOGjZAfd', function=Function(arguments='{"source_id": "20240813150354universidadexternadoleccionesdederechopenalparteespecialvol12011pdf_chunk616"}', name='get_next_chunk'), type='function'), ChatCompletionMessageToolCall(id='call_4iTXZe4GZNvcxKYgaZDi5W3y', function=Function(arguments='{"source_id": "20240825062302csjsentencia2749527062007pdf_chunk3"}', name='get_next_chunk'), type='function')]), content_filter_results={})], created=1725983361, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_80a1bad4c7', usage=CompletionUsage(completion_tokens=88, prompt_tokens=3506, total_tokens=3594), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'fi

In [376]:
def get_next_chunk_func(id):
    pattern = r"^(.*_chunk)(\d+)$"
    match = re.search(pattern, id)

    if match:
        prefix = match.group(1)
        chunk_number = int(match.group(2))
        return search_client.search(
            filter=f"id eq '{prefix}{chunk_number + 1}' or id eq '{prefix}{chunk_number + 2}'",
            top=2,
        )

In [377]:
third_completion = ""

if second_completion.choices[0].finish_reason == "tool_calls":
    tool_calls = second_completion.choices[0].message.tool_calls
    new_docs = []
    chunks_ids = ""
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        args = json.loads(
            second_completion.choices[0].message.tool_calls[0].function.arguments
        )
        documents = []
        if tool_name == "get_next_chunk":
            chunk_id = args.get("source_id")
            chunks_ids += chunk_id + ","
            next_chunks = list(get_next_chunk_func(chunk_id))
            documents += next_chunks
        elif tool_name == "search_legal_info":
            print("Search mock...")

        for document in documents:
            doc_formatted = {
                # "score": document["@search.score"],
                # "rerank": document["@search.reranker_score"],
                # "captions": captions_text,
                "id": document["id"],
                "título": document["title"],
                "autor": document["author"],
                "página": document["page"],
                "tema_legal": document["keywords"],
                "tipo_de_documento": document["category"],
                "contenido": document["content"],
            }
            new_docs.append(doc_formatted)

    print("New Docs:")
    print(json.dumps(new_docs, indent=4))
    
    new_document_stack = reranked_documents + new_docs

    ### 🤖 Generar respuesta
    new_messages = [
    {"role": "system", "content": response_system_template},
    {"role": "user", "content": user_input},
    {
        "role": "assistant",
        "content": f"Chunks encontrados al consultar con el query '{query.get("search_query")}': \n\n {reranked_documents}",
    },
    {
        "role": "assistant",
        "content": f"Chunks siguientes a '{chunks_ids}': \n\n {new_docs}",
    },
    ]
    
    
    print("========================")
    print("Chunks: ", len(new_document_stack))
    print("Messages Array:")
    print(json.dumps(new_messages, indent=4))

    third_completion = openai_client.chat.completions.create(
        tools=[search_legal_info, get_next_chunk],
        messages=new_messages,
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        temperature=0.2,
        n=1,
        tool_choice="auto",
    )

New Docs:
[
    {
        "id": "20240813150354universidadexternadoleccionesdederechopenalparteespecialvol12011pdf_chunk618",
        "t\u00edtulo": "UNIVERSIDAD EXTERNADO - Lecciones de Derecho penal - parte especial vol 1 -2011",
        "autor": "Universidad Externado",
        "p\u00e1gina": 538,
        "tema_legal": "Penal",
        "tipo_de_documento": "Doctrina",
        "contenido": "cuerpo objetivo del documento. De otra parte, reafirma lo dicho el hecho de\nque la ley presuma la autenticidad de los documentos privados que se\npresentan al proceso con fines probatorios-{-4-14}.\nLa idea en alg\u00fan sector de que la autenticidad da m\u00e1s fuerza probatoria al\ndocumento es completamente errada, puesto que la fuerza probante del\ndocumento se deriva del documento mismo: el ser aut\u00e9ntico (no el estar\nautenticado), qui\u00e9n es su autor, y la forma del contenido representativo o\ndeclarativo, que son la materia de quien valora la prueba. Pero, adem\u00e1s, para\nefecto

In [378]:
### 💬 Formatear respuesta

print(third_completion)
print("===========")
print(third_completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-A5xPHuWhE2ThV803N4uyAydwCCTvF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<strong>Delito de falsificación de documento privado</strong>\n\nSegún el artículo 289 del Código Penal Colombiano, la falsificación de documento privado es un delito que se configura cuando una persona falsifica un documento privado con el propósito de usarlo como prueba en el tráfico jurídico. Este delito se agrava si el documento falsificado es utilizado para obtener un beneficio o causar un perjuicio [UNIVERSIDAD EXTERNADO - Lecciones de Derecho penal - parte especial vol 1 -2011, p. 537] [CSJ - Sentencia 27495(27-06-2007)].\n\n<strong>Uso del documento falsificado</strong>\n\nEl uso de un documento falsificado también constituye un delito. En el caso de que A falsifique un cheque y lo utilice para comprar un televisor a B, A estaría cometiendo el delito de falsificación de documento privado y, adicionalmente, el delito de 